In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import click
import numpy as np
import pandas as pd
import random
from pathlib import Path

!pip install scapy
from scapy.compat import raw
from scapy.layers.inet import IP, UDP
from scapy.layers.l2 import Ether
from scapy.packet import Padding
from scipy import sparse
from scapy.layers.dns import DNS
from scapy.layers.inet import TCP
from scapy.packet import Padding
from scapy.utils import rdpcap

     |████████████████████████████████| 1.0MB 7.0MB/s 
  Created wheel for scapy: filename=scapy-2.4.4-py2.py3-none-any.whl size=1189182 sha256=b011059120ea392a12f3ad7b48aefd43b73384a565498692e53d3e29c8878f67
  Stored in directory: /root/.cache/pip/wheels/2c/e7/01/f097df99ac9cd0d4f744c255f918d471d7a4b0766bc84c38c3
Successfully built scapy


based on munhouiani's GitHub repository (https://github.com/munhouiani/Deep-Packet)

In [ ]:
# for app identification
PREFIX_TO_APP_ID = {
    # AIM chat
    'aim_chat_3a': 0,
    'aim_chat_3b': 0,
    'aimchat1': 0,
    'aimchat2': 0,
    # Email
    'email1a': 1,
    'email1b': 1,
    'email2a': 1,
    'email2b': 1,
    # Facebook
    'facebook_audio1a': 2,
    'facebook_audio1b': 2,
    'facebook_audio2a': 2,
    'facebook_audio2b': 2,
    'facebook_audio3': 2,
    'facebook_audio4': 2,
    'facebook_chat_4a': 2,
    'facebook_chat_4b': 2,
    'facebook_video1a': 2,
    'facebook_video1b': 2,
    'facebook_video2a': 2,
    'facebook_video2b': 2,
    'facebookchat1': 2,
    'facebookchat2': 2,
    'facebookchat3': 2,
    # FTPS
    'ftps_down_1a': 3,
    'ftps_down_1b': 3,
    'ftps_up_2a': 3,
    'ftps_up_2b': 3,
    # Gmail
    'gmailchat1': 4,
    'gmailchat2': 4,
    'gmailchat3': 4,
    # Hangouts
    'hangout_chat_4b': 5,
    'hangouts_audio1a': 5,
    'hangouts_audio1b': 5,
    'hangouts_audio2a': 5,
    'hangouts_audio2b': 5,
    'hangouts_audio3': 5,
    'hangouts_audio4': 5,
    'hangouts_chat_4a': 5,
    'hangouts_video1b': 5,
    'hangouts_video2a': 5,
    'hangouts_video2b': 5,
    # ICQ
    'icq_chat_3a': 6,
    'icq_chat_3b': 6,
    'icqchat1': 6,
    'icqchat2': 6,
    # Netflix
    'netflix1': 7,
    'netflix2': 7,
    'netflix3': 7,
    'netflix4': 7,
    # SCP
    'scp1': 8,
    'scpdown1': 8,
    'scpdown2': 8,
    'scpdown3': 8,
    'scpdown4': 8,
    'scpdown5': 8,
    'scpdown6': 8,
    'scpup1': 8,
    'scpup2': 8,
    'scpup3': 8,
    'scpup5': 8,
    'scpup6': 8,
    # SFTP
    'sftp1': 9,
    'sftp_down_3a': 9,
    'sftp_down_3b': 9,
    'sftp_up_2a': 9,
    'sftp_up_2b': 9,
    'sftpdown1': 9,
    'sftpdown2': 9,
    'sftpup1': 9,
    # Skype
    'skype_audio1a': 10,
    'skype_audio1b': 10,
    'skype_audio2a': 10,
    'skype_audio2b': 10,
    'skype_audio3': 10,
    'skype_audio4': 10,
    'skype_chat1a': 10,
    'skype_chat1b': 10,
    'skype_file1': 10,
    'skype_file2': 10,
    'skype_file3': 10,
    'skype_file4': 10,
    'skype_file5': 10,
    'skype_file6': 10,
    'skype_file7': 10,
    'skype_file8': 10,
    'skype_video1a': 10,
    'skype_video1b': 10,
    'skype_video2a': 10,
    'skype_video2b': 10,
    # Spotify
    'spotify1': 11,
    'spotify2': 11,
    'spotify3': 11,
    'spotify4': 11,
    # Torrent
    'torrent01': 12,
    # Tor
    'torfacebook': 13,
    'torgoogle': 13,
    'tortwitter': 13,
    'torvimeo1': 13,
    'torvimeo2': 13,
    'torvimeo3': 13,
    'toryoutube1': 13,
    'toryoutube2': 13,
    'toryoutube3': 13,
    # Vimeo
    'vimeo1': 14,
    'vimeo2': 14,
    'vimeo3': 14,
    'vimeo4': 14,
    # Voipbuster
    'voipbuster1b': 15,
    'voipbuster2b': 15,
    'voipbuster3b': 15,
    'voipbuster_4a': 15,
    'voipbuster_4b': 15,
    # Youtube
    'youtube1': 16,
    'youtube2': 16,
    'youtube3': 16,
    'youtube4': 16,
    'youtube5': 16,
    'youtube6': 16,
    'youtubehtml5_1': 16,
}

ID_TO_APP = {
    0: 'AIM Chat', #
    1: 'Email', #
    2: 'Facebook', #
    3: 'FTPS', #
    4: 'Gmail',
    5: 'Hangouts', #
    6: 'ICQ',
    7: 'Netflix',
    8: 'SCP', #
    9: 'SFTP', #
    10: 'Skype', #
    11: 'Spotify',
    12: 'Torrent',
    13: 'Tor', #
    14: 'Vimeo',
    15: 'Voipbuster',
    16: 'Youtube', #
}

# for traffic identification
PREFIX_TO_TRAFFIC_ID = {
    # Chat
    'aim_chat_3a': 0,
    'aim_chat_3b': 0,
    'aimchat1': 0,
    'aimchat2': 0,
    'facebook_chat_4a': 0,
    'facebook_chat_4b': 0,
    'facebookchat1': 0,
    'facebookchat2': 0,
    'facebookchat3': 0,
    'hangout_chat_4b': 0,
    'hangouts_chat_4a': 0,
    'icq_chat_3a': 0,
    'icq_chat_3b': 0,
    'icqchat1': 0,
    'icqchat2': 0,
    'skype_chat1a': 0,
    'skype_chat1b': 0,
    # Email
    'email1a': 1,
    'email1b': 1,
    'email2a': 1,
    'email2b': 1,
    # File Transfer
    'ftps_down_1a': 2,
    'ftps_down_1b': 2,
    'ftps_up_2a': 2,
    'ftps_up_2b': 2,
    'sftp1': 2,
    'sftp_down_3a': 2,
    'sftp_down_3b': 2,
    'sftp_up_2a': 2,
    'sftp_up_2b': 2,
    'sftpdown1': 2,
    'sftpdown2': 2,
    'sftpup1': 2,
    'skype_file1': 2,
    'skype_file2': 2,
    'skype_file3': 2,
    'skype_file4': 2,
    'skype_file5': 2,
    'skype_file6': 2,
    'skype_file7': 2,
    'skype_file8': 2,
    # Streaming
    'vimeo1': 3,
    'vimeo2': 3,
    'vimeo3': 3,
    'vimeo4': 3,
    'youtube1': 3,
    'youtube2': 3,
    'youtube3': 3,
    'youtube4': 3,
    'youtube5': 3,
    'youtube6': 3,
    'youtubehtml5_1': 3,
    # Torrent
    'torrent01': 4,
    # VoIP
    'facebook_audio1a': 5,
    'facebook_audio1b': 5,
    'facebook_audio2a': 5,
    'facebook_audio2b': 5,
    'facebook_audio3': 5,
    'facebook_audio4': 5,
    'hangouts_audio1a': 5,
    'hangouts_audio1b': 5,
    'hangouts_audio2a': 5,
    'hangouts_audio2b': 5,
    'hangouts_audio3': 5,
    'hangouts_audio4': 5,
    'skype_audio1a': 5,
    'skype_audio1b': 5,
    'skype_audio2a': 5,
    'skype_audio2b': 5,
    'skype_audio3': 5,
    'skype_audio4': 5,
    # VPN: Chat
    'vpn_aim_chat1a': 6,
    'vpn_aim_chat1b': 6,
    'vpn_facebook_chat1a': 6,
    'vpn_facebook_chat1b': 6,
    'vpn_hangouts_chat1a': 6,
    'vpn_hangouts_chat1b': 6,
    'vpn_icq_chat1a': 6,
    'vpn_icq_chat1b': 6,
    'vpn_skype_chat1a': 6,
    'vpn_skype_chat1b': 6,
    # VPN: File Transfer
    'vpn_ftps_a': 7,
    'vpn_ftps_b': 7,
    'vpn_sftp_a': 7,
    'vpn_sftp_b': 7,
    'vpn_skype_files1a': 7,
    'vpn_skype_files1b': 7,
    # VPN: Email
    'vpn_email2a': 8,
    'vpn_email2b': 8,
    # VPN: Streaming
    'vpn_vimeo_a': 9,
    'vpn_vimeo_b': 9,
    'vpn_youtube_a': 9,
    # VPN: Torrent
    'vpn_bittorrent': 10,
    # VPN VoIP
    'vpn_facebook_audio2': 11,
    'vpn_hangouts_audio1': 11,
    'vpn_hangouts_audio2': 11,
    'vpn_skype_audio1': 11,
    'vpn_skype_audio2': 11,
}

ID_TO_TRAFFIC = {
    0: 'Chat',
    1: 'Email',
    2: 'File Transfer',
    3: 'Streaming',
    4: 'Torrent',
    5: 'Voip',
    6: 'VPN: Chat',
    7: 'VPN: File Transfer',
    8: 'VPN: Email',
    9: 'VPN: Streaming',
    10: 'VPN: Torrent',
    11: 'VPN: Voip',
}


In [ ]:
def read_pcap(path: Path):
    packets = rdpcap(str(path))
    return packets


def should_omit_packet(packet):
    # SYN, ACK or FIN flags set to 1 and no payload
    if TCP in packet and (packet.flags & 0x13):
        # not payload or contains only padding
        layers = packet[TCP].payload.layers()
        if not layers or (Padding in layers and len(layers) == 1):
            return True

    # DNS segment
    if DNS in packet:
        return True

    return False


def remove_ether_header(packet):
    if Ether in packet:
        return packet[Ether].payload
    return packet


def mask_ip(packet):
    if IP in packet:
        packet[IP].src = '0.0.0.0'
        packet[IP].dst = '0.0.0.0'

    return packet


def pad_udp(packet):
    if UDP in packet:
        # get layers after udp
        layer_after = packet[UDP].payload.copy()

        # build a padding layer
        pad = Padding()
        pad.load = '\x00' * 12

        layer_before = packet.copy()
        layer_before[UDP].remove_payload()
        packet = layer_before / pad / layer_after

        return packet

    return packet

original, trimmed and sampled packets

In [ ]:
def packet_to_sparse_array(packet, max_length=1500):
    # Original implementation
    #print('raw packet\n', raw(packet))
    arr = np.frombuffer(raw(packet), dtype=np.uint8)[0: max_length] / 255  # divide each byte by 255
    #print('arr before padding\n', arr)
    if len(arr) < max_length:
        pad_width = max_length - len(arr)
        arr = np.pad(arr, pad_width=(0, pad_width), constant_values=0)  # padding the packet
        #print('arr after padding\n', arr)

    arr = sparse.csr_matrix(arr)
    return arr


def trimmed_start_packet_to_sparse_array(packet, trimmed_length):
    # Basic compressing method - trimming the packet (first X)
    #print('raw packet\n', raw(packet))
    arr = np.frombuffer(raw(packet), dtype=np.uint8)[:] / 255
    #print('arr before trimming\n', arr)
    arr = arr[:trimmed_length]  # trimming the packet
    #print('arr after trimming\n', arr)

    arr = sparse.csr_matrix(arr)
    return arr

def trimmed_finish_packet_to_sparse_array(packet, trimmed_length):
    # Basic compressing method - trimming the packet (last X)
    #print('raw packet\n', raw(packet))
    arr = np.frombuffer(raw(packet), dtype=np.uint8)[:] / 255
    #print('arr before trimming\n', arr)
    arr = arr[-1 * trimmed_length:-1]  # trimming the packet
    #print('arr after trimming\n', arr)

    arr = sparse.csr_matrix(arr)
    return arr


def sampled_packet_to_sparse_array(packet, sampled_length):
    # Using reservoir sampling
    #print('raw packet\n', raw(packet))
    arr = np.frombuffer(raw(packet), dtype=np.uint8)[:] / 255
    #print('arr before sampling\n', arr)

    # reservoir sampling
    i = 0
    r = np.zeros(sampled_length)
    for i in range(sampled_length):
        r[i] = arr[i]
    for i in range(len(arr)):
        j = random.randrange(i + 1)
        if j < sampled_length:
            k = random.randrange(sampled_length)
            r[k] = arr[i]
    #print('arr after sampling\n', r)

    r = sparse.csr_matrix(r)
    return r

create source-destination sparse matrix (for matrix completion)

In [ ]:
def create_src_dst_matrix(path):
    data = []
    for packet in read_pcap(path):
        if IP in packet:
            data.append([packet[IP].src, packet[IP].dst])
    df = pd.DataFrame(data, columns=['source', 'destination'])
    df = df.groupby(df.columns.tolist(), as_index=False).size()
    df = df.pivot(index='source', columns='destination', values='size')
    df.fillna(0, inplace=True)

    return df

final preprocessing - packet transformation

In [ ]:
def transform_packet(packet):
    if should_omit_packet(packet):
        return None
    #print('packet before delete\n', packet)
    packet = remove_ether_header(packet)
    packet = pad_udp(packet)
    packet = mask_ip(packet)
    #print('packet after delete\n', packet)

    arr = packet_to_sparse_array(packet) # if original implementation (original packet length)
    #arr = trimmed_start_packet_to_sparse_array(arr, arr.shape[1]) # if trimmed packet start
    #arr = trimmed_finish_packet_to_sparse_array(arr, arr.shape[1]) # if trimmed packet finish
    #arr = sampled_packet_to_sparse_array(arr, arr.shape[1]) # if sampled packet

    return arr


def transform_pcap(path, output_path: Path = None, output_batch_size=10000):
    if Path(str(output_path.absolute()) + '_SUCCESS').exists():
        print(output_path, 'Done')
        return

    print('Processing', path)

    rows = []
    batch_index = 0
    for i, packet in enumerate(read_pcap(path)):
        arr = transform_packet(packet)
        if arr is not None:
            # get labels for app identification
            # print(arr)
            prefix = path.name.split('.')[0].lower()
            app_label = PREFIX_TO_APP_ID.get(prefix)
            traffic_label = PREFIX_TO_TRAFFIC_ID.get(prefix)
            row = {
                'app_label': app_label,
                'traffic_label': traffic_label,
                'feature': arr.todense().tolist()[0]
            }
            # print(row)
            rows.append(row)

        # write every batch_size packets, by default 10000
        if rows and i > 0 and i % output_batch_size == 0:
            #part_output_path = Path(str(output_path.absolute()) + f'_part_{batch_index:04d}.parquet')
            part_output_path = Path(str(output_path.absolute()) + f'_part_{batch_index:04d}.csv')
            df = pd.DataFrame(rows)
            df.to_csv(part_output_path)
            batch_index += 1
            rows.clear()

    # final write
    if rows:
        df = pd.DataFrame(rows)
        # part_output_path = Path(str(output_path.absolute()) + f'_part_{batch_index:04d}.parquet')
        part_output_path = Path(str(output_path.absolute()) + f'_part_{batch_index:04d}.csv')
        df.to_csv(part_output_path)

    # write success file
    with Path(str(output_path.absolute()) + '_SUCCESS').open('w') as f:
        f.write('')

    print(output_path, 'Done')

In [ ]:
@click.command()
@click.option('-s', '--source', help='path to the directory containing raw pcap files', required=True)
@click.option('-t', '--target', help='path to the directory for persisting preprocessed files', required=True)
def main(source, target):
    data_dir_path = Path(source)
    target_dir_path = Path(target)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    for pcap_path in sorted(data_dir_path.iterdir()):
        transform_pcap(pcap_path, target_dir_path / (pcap_path.name + '.transformed'))
        src_dst = create_src_dst_matrix(pcap_path)


if __name__ == '__main__':
    main()

In [ ]:
def main(source, target):
    data_dir_path = ('/content/drive/MyDrive/')
    target_dir_path = Path(target)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    for pcap_path in sorted(data_dir_path.iterdir()):
        transform_pcap(pcap_path, target_dir_path / (pcap_path.name + '.transformed'))
        src_dst = create_src_dst_matrix(pcap_path)


if __name__ == '__main__':
    main()